### This notebook contains the script that was used for allocating Uniprot ID to the NE proteins reported in the 4 proteome papers:
### Schirmer et al. 2003; Korfali et al. 2010, 2012; Wilkie et al. 2012
### Some processes such as information extraction from the original source table and NCBI ID conversion were performed outside this script

In [38]:
import os, re, requests, json, openpyxl, warnings, importlib
import PyPDF2
import numpy as np
import pandas as pd
from urllib.request import urlopen
from time import sleep

from my_utils import *

warnings.filterwarnings('ignore')

# Schirmer et al 2003: mouse liver

## 1. Import the orginal data file PDF

In [2]:
directory = './SourceData/Schirmer2003/'
filename = 'schirmer.som.table_S8.pdf'
pdf = open(directory + filename, 'rb')

## 2. Read the pdf and extract the whole text

In [3]:
pdf_reader = PyPDF2.PdfFileReader(pdf)
pdf_reader.numPages

15

In [4]:
whole_txt = ''
for page_num in range(pdf_reader.numPages):
    pdf_page = pdf_reader.getPage(page_num)
    pdf_page_txt = pdf_page.extractText()
    whole_txt += pdf_page_txt

## 3. Extract NCBI IDs by regex

In [13]:
# put ref| as a marker of where id is
# Either NP or XP is used
# version number e.g. ".1" can be absent by putting ?

# this is the intitial regex that inculdes the version number
# but realized that inclusion of the version number could hamper ID conversion to Uniprot ID
# regex = re.compile(r'(ref\|)(NP_\d+\.?\d?|XP_\d+\.?\d?)')

# therefore instead extracted only the main body without the version number
regex = re.compile(r'(ref\|)(NP_\d+|XP_\d+)')

In [14]:
# Using regex, extract the IDs from the text and put them to a list
ncbi_id_list = []
for groups in regex.findall(whole_txt):
    ncbi_id = groups[1]
    ncbi_id_list.append(ncbi_id)

In [15]:
len(ncbi_id_list)

109

In [8]:
# below to make sure the number of ids is correct
# split the text to each gene desription using ">gi" as a sign
whole_list = whole_txt.replace('\n', '').split('>gi')
len(whole_list)

110

#### As seen below the first in the "whole_list" is not a protein record. Thus 109 is correct

In [9]:
whole_list[:2]

['Table S8. Amino acid sequences of the putative nuclear transmembrane proteins in fasta format',
 '|7661996|ref|NP_055688.1| KIAA0205 gene product [Homo sapiens]MAITLEEAPWLGWLLVKALMRFAFMVVNNLVAIPSYICYVIILQPLRVLDSKRFWYIEGIMYKWLLGMVASWGWYAGYTVMEWGEDIKAVSKDEAVMLVNHQATGDVCTLMMCLQDKGLVVAQMMWLMDHIFKYTNFGIVSLVHGDFFIRQGRSYRDQQLLLLKKHLENNYRSRDRKWIVLFPEGGFLRKRRETSQAFAKKNNLPFLTNVTLPRSGATKIILNALVAQQKNGSPAGGDAKELDSKSKGLQWIIDTTIAYPKAEPIDIQTWILGYRKPTVTHVHYRIFPIKDVPLETDDLTTWLYQRFVEKEDLLSHFYETGAFPPSKGHKEAVSREMTLSNLWIFLIQSFAFLSGYMWYNIIQYFYHCLF']

## 4. Translate the NCBI list to Uniprot ID

In [17]:
' '.join(ncbi_id_list)

'NP_055688 NP_056473 NP_057086 NP_060531 NP_060557 NP_659471 NP_060221 XP_087089 NP_061900 XP_298567 NP_057688 NP_061891 XP_166338 NP_116231 XP_291222 NP_116117 NP_775961 NP_056348 NP_775857 NP_060634 NP_009107 XP_027330 NP_056158 NP_060640 XP_292696 NP_067038 XP_129726 XP_129627 XP_129651 NP_766431 XP_130243 XP_130605 NP_694766 XP_131120 XP_131247 XP_283952 XP_149561 NP_064382 NP_082631 NP_081113 NP_083003 NP_067278 XP_110572 NP_705798 NP_084313 XP_194313 XP_285910 NP_620096 XP_163618 NP_080922 XP_150109 NP_705820 NP_080950 XP_125636 XP_137321 XP_125972 NP_080293 NP_081625 NP_067421 NP_766088 XP_127498 NP_079663 NP_083179 NP_666167 XP_127968 XP_128377 NP_084221 NP_666084 XP_128762 NP_080446 NP_666187 XP_128954 NP_659110 XP_129087 NP_663471 NP_766428 XP_219484 XP_226713 XP_226886 XP_227591 XP_230334 XP_230798 XP_231194 XP_231568 XP_232987 XP_233684 XP_233702 XP_233726 XP_234304 XP_216756 XP_234559 XP_234768 XP_234929 XP_235233 XP_235527 XP_243957 XP_213272 XP_213339 XP_221193 XP_221266

#### Copied above and coverted them to Uniprot ID on Retrieve/ID mapping (3/20/22)
#### Obtained mapped and unmapped IDs, each of which saved as a XLSX or CSV

In [5]:
df_mapped = pd.read_excel('./SourceData/Schirmer2003/Mapped.xlsx')
df_unmapped = pd.read_csv('./SourceData/Schirmer2003/Unmapped.csv')

In [6]:
df_mapped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62 entries, 0 to 61
Data columns (total 7 columns):
 #   Column                                                     Non-Null Count  Dtype 
---  ------                                                     --------------  ----- 
 0   yourlist:M202203204ABAA9BC7178C81CEBC9459510EDDEA346CE44X  62 non-null     object
 1   isomap:M202203204ABAA9BC7178C81CEBC9459510EDDEA346CE44X    19 non-null     object
 2   Entry                                                      62 non-null     object
 3   Entry name                                                 62 non-null     object
 4   Protein names                                              62 non-null     object
 5   Gene names                                                 58 non-null     object
 6   Organism                                                   62 non-null     object
dtypes: object(7)
memory usage: 3.5+ KB


In [7]:
df_unmapped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61 entries, 0 to 60
Data columns (total 1 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   not mapped  61 non-null     object
dtypes: object(1)
memory usage: 620.0+ bytes


#### Quick check of what the unmapped ids look like

In [28]:
unmapped_ncbi_ids = list(df_unmapped['not mapped'])
unmapped_ncbi_ids[2:10]

['XP_166338',
 'XP_291222',
 'XP_027330',
 'XP_292696',
 'XP_129726',
 'XP_129627',
 'XP_129651',
 'XP_130243']

#### It seems like unmapped IDs are 'obsolete' on NCBI (3/20/22)
#### Thus for now it seems unproductive to further look into this list
#### This could mean the number of the NE proteins found here ends up with far less than 67 as proposed by the paper
#### After all it has been 19 years since then....

#### Let us shift gears and focus on mapped IDs

In [8]:
# drop the unnecessary column
df_mapped = df_mapped.drop(['isomap:M202203204ABAA9BC7178C81CEBC9459510EDDEA346CE44X'], axis=1)

# rename to NCBI ID
df_mapped = df_mapped.rename(columns={'yourlist:M202203204ABAA9BC7178C81CEBC9459510EDDEA346CE44X': 'NCBI ID'})

# remove duplicate
# Entry or Uniprot ID turns out to have no duplicate
# df_mapped_unique = df_mapped.drop_duplicates(subset=['Entry'])
df_mapped_unique = df_mapped.drop_duplicates(subset=['Protein names'])

In [9]:
df_mapped_unique.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 56 entries, 0 to 61
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   NCBI ID        56 non-null     object
 1   Entry          56 non-null     object
 2   Entry name     56 non-null     object
 3   Protein names  56 non-null     object
 4   Gene names     53 non-null     object
 5   Organism       56 non-null     object
dtypes: object(6)
memory usage: 3.1+ KB


In [10]:
df_mapped_unique.head()

,NCBI ID,Entry,Entry name,Protein names,Gene names,Organism
0,NP_055688,Q92604,LGAT1_HUMAN,Acyl-CoA:lysophosphatidylglycerol acyltransfer...,LPGAT1 FAM34A KIAA0205,Homo sapiens (Human)
1,NP_056473,Q9Y3T9,NOC2L_HUMAN,Nucleolar complex protein 2 homolog (Protein N...,NOC2L NIR,Homo sapiens (Human)
2,NP_057086,Q8NBX0,SCPDL_HUMAN,Saccharopine dehydrogenase-like oxidoreductase...,SCCPDH CGI-49,Homo sapiens (Human)
3,NP_057086,A0A384NPM7,A0A384NPM7_HUMAN,Epididymis secretory sperm binding protein (Sa...,SCCPDH hCG_1782151,Homo sapiens (Human)
4,NP_060531,Q5VTL8,PR38B_HUMAN,Pre-mRNA-splicing factor 38B (Sarcoma antigen ...,PRPF38B,Homo sapiens (Human)


In [11]:
df_mapped_unique.tail()

,NCBI ID,Entry,Entry name,Protein names,Gene names,Organism
55,NP_080446,Q4FK22,Q4FK22_MOUSE,1200007D18Rik protein (MAA-136 protein),Ergic1 maa-136,Mus musculus (Mouse)
56,NP_666187,Q6DVA0,LEMD2_MOUSE,LEM domain-containing protein 2 (Nuclear envel...,Lemd2 Lem2,Mus musculus (Mouse)
59,NP_766428,Q70UZ7,VWA2_MOUSE,von Willebrand factor A domain-containing prot...,Vwa2 Amaco,Mus musculus (Mouse)
60,NP_766428,Q8CE01,Q8CE01_MOUSE,Uncharacterized protein,Vwa2 Amaco,Mus musculus (Mouse)
61,XP_233702,G3V7B0,G3V7B0_RAT,Nucleolar protein 9,Nol9 rCG_31415,Rattus norvegicus (Rat)


In [12]:
df_mapped_unique.Organism.unique()

array(['Homo sapiens (Human)', 'Mus musculus (Mouse)',
       'Rattus norvegicus (Rat)'], dtype=object)

## 5. Convert mouse and rat Entry to humans

In [4]:
df_mapped_unique_MR = df_mapped_unique[df_mapped_unique.Organism != 'Homo sapiens (Human)']

In [49]:
# uniprot API URL
WEBSITE_API = "https://rest.uniprot.org/uniprotkb"

# organism id
organism_id_list = {'Homo sapiens': '9606'}
organism_id = organism_id_list['Homo sapiens']

my_dict = {'entry_mr': [], 'gene_name': [], 'entry_h': []}

for entry in df_mapped_unique_MR.Entry:
    try:
        # get response with the query, then the gene name in it
        r = get_url(f'{WEBSITE_API}/search?query=accession:{entry}&fields=gene_names')
        gene_name = r.json()['results'][0]['genes'][0]['geneName']['value']
        sleep(1)

        # find the entry of the human version of the gene
        r2 = get_url(f'{WEBSITE_API}/search?query=gene:{gene_name}+AND+organism_id:{organism_id}&fields=accession,gene_names')
        entry_hs = r2.json()['results'][0]['primaryAccession']
        sleep(1)

    except:
        gene_name = 'Not_found'
        entry_hs = 'Not_found'
    
    my_dict['entry_mr'].append(entry)
    my_dict['gene_name'].append(gene_name)
    my_dict['entry_h'].append(entry_hs)
    

In [59]:
_df = pd.DataFrame(my_dict)
_df = _df.drop_duplicates(subset='entry_h')
_df = _df[_df.entry_h != 'Not_found']
_df = _df.drop(columns=['entry_mr'])
_df = _df.rename(columns={'gene_name':'Gene names', 'entry_h':'Entry'})
_df.head()

,Gene names,Entry
0,Tor1aip2,Q8NFQ8
1,Nat10,Q9H0A0
2,Alg2,O75340
3,Ndc1,Q9BTX1
4,Tmem53,Q6P2H8


#### Merge with the human entries from df_mapped

In [61]:
df_mapped_unique_Hs = df_mapped_unique[df_mapped_unique.Organism == 'Homo sapiens (Human)']
df_mapped_unique_Hs = df_mapped_unique_Hs[['Entry', 'Gene names']]
df = pd.concat([df_mapped_unique_Hs, _df])

In [66]:
df = df.dropna(axis=0)

In [67]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44 entries, 0 to 27
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Entry       44 non-null     object
 1   Gene names  44 non-null     object
dtypes: object(2)
memory usage: 1.0+ KB


#### Export

In [68]:
df.to_csv('./Output/Schirmer2003/Output.csv', index=False)

# Korfali et al 2010: Mouse leukocytes

In [46]:
# Excel import
df_Korfali_2010 = pd.read_excel('./SourceData/Korfali_2010/rk1-139_fromTableS5.xlsx') # gene names were manually extracted in this file beforehand

# Convert to df
df_Korfali_2010= pd.DataFrame(df_Korfali_2010.iloc[1:, 1])
df_Korfali_2010.columns = ['Gene_name']
df_Korfali_2010 = df_Korfali_2010.reset_index(drop=True)

In [47]:
# Manual inspection and correction of gene names
df_Korfali_2010.iloc[15, 0] = 'DHRS7'
df_Korfali_2010.iloc[39, 0] = 'NOC2L'
df_Korfali_2010.iloc[58, 0] = 'METTL7A'
df_Korfali_2010.iloc[104, 0] = 'TMEM189'

In [48]:
# find UniProt Entry and UniProt-registerd gene names
for i in range(len(df_Korfali_2010)):
    
    gene = df_Korfali_2010.iloc[i, 0]
    entry_converted, gene_obtained = get_UniProtEntry(gene)
    
    df_Korfali_2010.loc[i, 'Entry_Korfali_2010'] = entry_converted
    df_Korfali_2010.loc[i, 'Gene_name_obtained'] = gene_obtained
    
    if i % 40 == 0: print(i, entry_converted, gene_obtained)
    
    sleep(1)

0 Q9UH99 SUN2
40 Q3LXA3 TKFC
80 Q6NXT6 TAPT1
120 Q9Y2W6 TDRKH


In [16]:
# Clean and export
df_Korfali_2010 = df_Korfali_2010.drop(['Gene_name'], axis=1)
df_Korfali_2010.to_csv('./IntermediateProducts/ProteomePapers/Korfali_2010.csv', index=False)

# Korfali et al 2012: 

In [57]:
# S4 was extracted beforehand to a single file because the whole xlsx is extremely heavy
df_Korfali_2012 = pd.read_excel('./SourceData/Korfali_2012/2012NUCLEUS0047R-SupTables_S4.xlsx')

In [58]:
# prepare column names
columns = df_Korfali_2012.iloc[1,[0,1,2,3,17,18]]
columns

Table S4. Summary of NETs directly analyzed                  tissue
Unnamed: 1                                                gene name
Unnamed: 2                                          alternate names
Unnamed: 3                                        accession numbers
Unnamed: 17                                    NE:MM ratio by dNSAF
Unnamed: 18                                               reference
Name: 1, dtype: object

In [53]:
# Extract necessary cells
df_Korfali_2012 = df_Korfali_2012.iloc[2:136,[0,1,2,3,17,18]]

# rename the columns
df_Korfali_2012.columns = columns

# Fill in tissue data
df_Korfali_2012['tissue'] = df_Korfali_2012['tissue'].fillna(method='ffill')
df_Korfali_2012 = df_Korfali_2012.dropna(axis=0)

# Reindex
df_Korfali_2012 = df_Korfali_2012.reset_index(drop=True)

In [55]:
print("The number of genes: ", len(list(df_Korfali_2012['gene name'].unique())))

The number of genes:  119


In [27]:
# Manual inspection and correction of gene names
# #15 should be SLC22A24
# #18 contains two names and only the latter VMA21 is needed
# #47 should be MARCH5
df_Korfali_2012.iloc[14, 1] = 'SLC22A24'
df_Korfali_2012.iloc[18, 1] = 'VMA21'
df_Korfali_2012.iloc[47, 1] = 'MARCH5'
df_Korfali_2012.iloc[97, 1] = 'ATL3'

In [28]:
# select columns
df_Korfali_2012 = df_Korfali_2012[['tissue', 'gene name', 'NE:MM ratio by dNSAF']]

In [29]:
# find Uniprot Entry and UniProt-registered gene names
for i in range(len(df_Korfali_2012)):
    
    gene = df_Korfali_2012.iloc[i, 1]
    entry_converted, gene_obtained = get_UniProtEntry(gene)
    
    df_Korfali_2012.loc[i, 'Entry_Korfali_2012'] = entry_converted
    df_Korfali_2012.loc[i, 'Gene_name_obtained'] = gene_obtained
    
    sleep(1)

In [32]:
# Clean and export
df = df.drop(columns=['Gene_name_obtained'], axis=1)
df.to_csv('./IntermediateProducts/ProteomePapers/Korfali_2012.csv', index=False)

# Wilkie et al 2010: Rat skeletal muscle

In [43]:
# Excel import
wb = openpyxl.load_workbook('./SourceData/Wilkie_2010/mcp.M110.003129-6.xlsx')
print(wb.get_sheet_names())

# sheet selected
sheet = wb.get_sheet_by_name('Extracted') # gene names were manually extracted in this sheet beforehand
_df_wilkie = pd.DataFrame(sheet.values)

# convert to df
df_wilkie = pd.DataFrame(_df_wilkie.iloc[3:, 1])
df_wilkie.columns = ['Gene_name']
df_wilkie = df_wilkie.reset_index(drop=True)
df_wilkie.head()

['Table 1', 'Extracted']


,Gene_name
0,UNC84B
1,TOR1AIP1
2,LBR
3,UNC84A
4,NUP210


In [44]:
# find UniProt Entry and UniProt-registered gene names
for i in range(len(df_wilkie)):
    
    gene = df_wilkie.iloc[i, 0]
    entry_converted, gene_obtained = get_UniProtEntry(gene)
    
    df_wilkie.loc[i, 'Entry_Wilkie'] = entry_converted
    df_wilkie.loc[i, 'Gene_name_obtained'] = gene_obtained
    
    sleep(1)

In [48]:
# Clean and export
df_wilkie = df_wilkie.drop(['Gene_name'], axis=1)
df_wilkie.to_csv('./IntermediateProducts/Wilkie_2010.csv', index=False)